In [5]:
# 📦 Install required packages (if needed)
!pip install pandas folium openrouteservice ipywidgets
from google.colab import output
output.enable_custom_widget_manager()

# 📚 Imports & Setup
import pandas as pd, folium, openrouteservice, ipywidgets as widgets
from IPython.display import display, HTML
from openrouteservice import convert

# 🔑 API key
api_key = 'eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6IjliNWQ5OWE0ZWY4MjRmN2VhMmZkZGVmZDY4NThiZTQyIiwiaCI6Im11cm11cjY0In0='
client = openrouteservice.Client(key=api_key)

# 🗂️ Load data
df = pd.read_csv("updated_cities_and_colleges.csv")
cities, colleges = df[df.Type == 'City'], df[df.Type == 'College']

# 💰 Fare logic
fare = lambda d: max(1.2 * d, 10)

# 🎛️ UI dropdowns
dropdowns = {
    'City': widgets.Dropdown(options=cities.Name.tolist(), description='City:', layout=widgets.Layout(width='50%')),
    'College': widgets.Dropdown(options=colleges.Name.tolist(), description='College:', layout=widgets.Layout(width='50%'))
}

# 🧭 Route & Map logic
def on_change(change=None):
    start, end = (df[df.Name == dropdowns['City'].value].iloc[0], df[df.Name == dropdowns['College'].value].iloc[0])
    coords = ((start.Longitude, start.Latitude), (end.Longitude, end.Latitude))

    try:
        r = client.directions(coords, profile='driving-car', format='geojson')
        geo, steps = r['features'][0]['geometry']['coordinates'], r['features'][0]['properties']['segments'][0]['steps']
        dist = r['features'][0]['properties']['summary']['distance'] / 1000
    except Exception as e:
        print("⚠️ Route error:", e)
        return

    print(f"\n🚏 {start.Name} ➡ {end.Name}\nDistance: {dist:.2f} km\nFare: ₹{fare(dist):.2f}\n📍 Directions:")
    for i, s in enumerate(steps, 1): print(f"{i}. {s['instruction']} ({s['distance']:.0f} m)")

    m = folium.Map(location=[(start.Latitude + end.Latitude)/2, (start.Longitude + end.Longitude)/2], zoom_start=12)
    for name, loc, color in [(start.Name, [start.Latitude, start.Longitude], 'blue'),
                             (end.Name, [end.Latitude, end.Longitude], 'green')]:
        folium.Marker(loc, popup=name, icon=folium.Icon(color=color)).add_to(m)

    folium.PolyLine([(y,x) for x,y in geo], color='red', weight=5).add_to(m)
    for i, s in enumerate(steps, 1):
        wp = geo[s['way_points'][0]]
        folium.Marker([wp[1], wp[0]], popup=f"{i}. {s['instruction']}", icon=folium.Icon(color='orange')).add_to(m)

    display(HTML(m._repr_html_()))

# 🚦 Activate
display(dropdowns['City'], dropdowns['College'])
dropdowns['City'].observe(on_change, names='value')
dropdowns['College'].observe(on_change, names='value')
on_change()

Dropdown(description='City:', layout=Layout(width='50%'), options=('Kakinada', 'Rajahmundry', 'Samalkot', 'Tun…

Dropdown(description='College:', layout=Layout(width='50%'), options=('Pragati Engineering College', 'Aditya E…


🚏 Kakinada ➡ Pragati Engineering College
Distance: 16.82 km
Fare: ₹20.18
📍 Directions:
1. Head northeast (44 m)
2. Turn left (197 m)
3. Turn right (532 m)
4. Turn left onto MDR (512 m)
5. Enter the roundabout and take the 3rd exit onto NH216 (3212 m)
6. Enter the roundabout and take the 2nd exit onto NH216 (9637 m)
7. Turn right onto Samarlakota - Uppada Road, SH173 (645 m)
8. Turn slight left onto Samarlakota - Uppada Road Pithapuram Main Road, SH173 (437 m)
9. Turn right onto MDR (1602 m)
10. Arrive at MDR, on the right (0 m)
